EVALUACIÓN MÓDULO 3 : PROBLEMA 2

Crea un conjunto de datos utilizando el siguiente código:

trX = np.linspace(-1, 1, 101)

trY = np.linspace(-1, 1, 101)

for i in range(len(trY)):

trY[i] = math.log(1 + 0.5 * abs(trX[i])) + trX[i] / 3 + np.random.randn() * 0.033

Ahora, utiliza Theano para obtener los parámetros w_0 y w_1 del siguiente modelo:

y = log(1+w0|x|)+w1*x

utilizando los datos generados anteriormente.

In [112]:
%pylab
%matplotlib inline

%config InlineBackend.figure_format = 'retina'

Using matplotlib backend: MacOSX
Populating the interactive namespace from numpy and matplotlib


/anaconda3/lib/python3.7/site-packages/IPython/core/magics/pylab.py:160: UserWarning: pylab import has clobbered these variables: ['f']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [115]:
import theano
import theano.tensor as T
from theano.ifelse import ifelse

Lo primero que hacemos en este código es un conjunto de datos de muestra como se pide en el enunciado. Para ello, se toman 101 puntos entre -1 y 1 para calcular el valor que tendría un modelo lineal para la expresión y = log(1+0,5|x|)+(1/3)*x ,a la que se le ha añadido un ruido blanco de amplitud y media igual a 0,033. Posteriormente, se crean dos variables para el modelo en el que se utilizan dos variables compartidas w0 y w1, una para cada uno de los parámetros del modelo.
Una vez se ha definido el modelo, calculamos la función de coste (cost) empleando el error cuadrático medio, para lo cual se ha de calcular el gradiente con respecto a los dos parámetros (gradient_w0 y gradient_w1) y se utilizan para definir una función de actualización. Esta actualización es el valor del parámetro menos el gradiente por el factor de aprendizaje, en nuestro código es de 0,01. Con todo esto, creamos una función Theano en la que se indiquen los valores de entrada, los de salida y la función de actualización.
En nuestro caso , realizamos el entrenamiento que se repite 101 veces y en la salida del código se puede ver que los valores se aproximan a los esperados , esto es , w0 próximo a 0,5 y w1 próximo a 1/3.

In [117]:
import numpy as np

trX = np.linspace(-1, 1, 101)

trY = np.linspace(-1, 1, 101)

for i in range(len(trY)):

    trY[i] = math.log(1 + 0.5 * abs(trX[i])) + trX[i] / 3 + np.random.randn() * 0.033

X = T.scalar()
Y = T.scalar()

def model(X, w0, w1):
    return log(1+w0*abs(X))+ w1*X

w0 = theano.shared(np.asarray(0., dtype = theano.config.floatX))
w1 = theano.shared(np.asarray(0., dtype = theano.config.floatX))
y = model(X, w0, w1)

cost     = T.mean(T.sqr(y - Y))
gradient_w0 = T.grad(cost = cost, wrt = w0)
gradient_w1 = T.grad(cost = cost, wrt = w1)
updates  = [[w0, w0 - gradient_w0 * 0.01], [w1, w1 - gradient_w1 * 0.01]]

train = theano.function(inputs = [X, Y], outputs = cost, updates = updates)

for i in range(101):
    for x, y in zip(trX, trY):
        cost_i = train(x, y)
    print('En el paso', i, 'el valor de w0 es', w0.get_value(),
           'y w1 es', w1.get_value(), 'con un coste', cost_i)

En el paso 0 el valor de w0 es 0.20180896897527895 y w1 es 0.17084543615893064 con un coste 0.16572786612579996
En el paso 1 el valor de w0 es 0.30847078616129053 y w1 es 0.257970449991101 con un coste 0.0521477531227361
En el paso 2 el valor de w0 es 0.37341689423186764 y w1 es 0.30068558068526396 con un coste 0.017972429794004974
En el paso 3 el valor de w0 es 0.4153331929118252 y w1 es 0.32112126547574 con un coste 0.006709864943645037
En el paso 4 el valor de w0 es 0.4431839553708907 y w1 es 0.3306223047711409 con un coste 0.002704914591998713
En el paso 5 el valor de w0 es 0.4619878682173289 y w1 es 0.3348545577850317 con un coste 0.001176374560387153
En el paso 6 el valor de w0 es 0.4748033263921529 y w1 es 0.33660497344315415 con un coste 0.0005522221540086622
En el paso 7 el valor de w0 es 0.48358757989043744 y w1 es 0.337224641770306 con un coste 0.0002802962935220404
En el paso 8 el valor de w0 es 0.4896303033157017 y w1 es 0.33735670685068736 con un coste 0.00015421975076414

Mediante una de las grandes características de Theano , como es la facilidad con la que se pueden calcular los gradientes de las funciones hemos podido obtener los parámetros del modelo utilizando el método del gradiente descendente (lo que permite obtener unos resultados tan buenos como podemos ver en los resultados y , tan solo con 9 ó 10 iteraciones).
Para calcular el gradiente de una variable respecto a otra, se ha de utilizar la función T.grad indicando como primer parámetro la variable sobre la que se desea estimar el gradiente y como segundo la variable respecto a la que se realiza. Esta función devuelve una nueva variable simbólica que se puede ser utilizada en las funciones o evaluada.La potencia de la función gradiente de Theano es tal que permite derivar cualquier tipo de función en polinomios, incluyendo funciones trigonométricas, logaritmos (como es nuestro caso ) y otras funciones más complejas.